In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize

# Define the base directory
base_dir = r"C:\Users\vaide\MITResearchWork\01032025_Gynepath_G2D51-20250624T155222Z-1-001"

# MRI data
mri_data = [
    {'Mouse': '3X', 'Uterus_Volume': 92600, 'Cyst_Volume': 4175, 'Cysts_to_Uterus': 4.51, 'LN_Volume': 6.90},
    {'Mouse': '3R', 'Uterus_Volume': 185800, 'Cyst_Volume': 36490, 'Cysts_to_Uterus': 19.64, 'LN_Volume': 3.54},
    {'Mouse': '3L', 'Uterus_Volume': 337200, 'Cyst_Volume': 103700, 'Cysts_to_Uterus': 30.75, 'LN_Volume': 3.05},
    {'Mouse': '3RR', 'Uterus_Volume': 128300, 'Cyst_Volume': 5762, 'Cysts_to_Uterus': 4.49, 'LN_Volume': 4.58},
    {'Mouse': '4R', 'Uterus_Volume': 91940, 'Cyst_Volume': 3735, 'Cysts_to_Uterus': 4.06, 'LN_Volume': 13.82},
    {'Mouse': '4L', 'Uterus_Volume': 95500, 'Cyst_Volume': 5212, 'Cysts_to_Uterus': 5.46, 'LN_Volume': 11.89},
    {'Mouse': '4RR', 'Uterus_Volume': 137200, 'Cyst_Volume': 31300, 'Cysts_to_Uterus': 22.81, 'LN_Volume': 9.25},
    {'Mouse': '4RL', 'Uterus_Volume': 75040, 'Cyst_Volume': 500, 'Cysts_to_Uterus': 0.67, 'LN_Volume': 10.08},
]

# Define subfolders
folders = [
    'GP20250103105329',  # Front, 3X, 3R, 3L, 3RR
    'GP20250103105531',  # Back, 3X, 3R, 3L, 3RR
    'GP20250103111346',  # Front, 4R, 4L, 4RR, 4RL
    'GP20250103111544'      # Back, 4R, 4L, 4RR, 4RL
]


# Order of mice in each image
mouse_order = [
    ['3X', '3R', '3L', '3RR'],
    ['3X', '3R', '3L', '3RR'],
    ['4R', '4L', '4RR', '4RL'],
    ['4R', '4L', '4RR', '4RL']
]



In [2]:
# Normalize utility
def normalize_image(im):
    return (im - np.min(im)) / (np.max(im) - np.min(im))

# Initialize luminescence per mouse
mouse_lumin = {mouse['Mouse']: 0 for mouse in mri_data}

In [4]:
# Collect luminescence from each folder
for folder in folders:
    full_path = os.path.join(base_dir, folder)
    if not os.path.isdir(full_path):
        print(f"Folder missing: {full_path}")
        continue

    for file in os.listdir(full_path):
        if file.lower().endswith('luminescent.tif'):
            parts = file.split('_')
            for mouse_id in mouse_lumin:
                if mouse_id in file:
                    img_path = os.path.join(full_path, file)
                    im = cv2.imread(img_path, -1).astype(np.float32)
                    mouse_lumin[mouse_id] += np.sum(normalize_image(im))
                    break

Folder missing: C:\Users\vaide\MITResearchWork\01032025_Gynepath_G2D51-20250624T155222Z-1-001\GP20250103111


In [5]:
# Attach luminescence data to MRI entries
for mouse in mri_data:
    lum = mouse_lumin.get(mouse['Mouse'], None)
    if lum == 0:
        print(f"[Warning] No IVIS data found for {mouse['Mouse']}")
        mouse['Luminescence_Sum'] = None
    else:
        mouse['Luminescence_Sum'] = lum

[Warning] No IVIS data found for 3X
[Warning] No IVIS data found for 3R
[Warning] No IVIS data found for 3L
[Warning] No IVIS data found for 3RR
[Warning] No IVIS data found for 4R
[Warning] No IVIS data found for 4L
[Warning] No IVIS data found for 4RR
[Warning] No IVIS data found for 4RL


In [16]:
# Plotting
mice_with_data = [m for m in mri_data if m['Luminescence_Sum'] is not None]
x = np.array([m['Cysts_to_Uterus'] for m in mice_with_data])
y = np.array([m['Luminescence_Sum'] for m in mice_with_data])

# Fit
slope, intercept = np.polyfit(x, y, 1)
x_fit = np.linspace(min(x), max(x), 100)
y_fit = slope * x_fit + intercept

# Plot
plt.figure(figsize=(8, 6))
sc = plt.scatter(x, y, c=x, cmap='viridis', s=120, edgecolor='black')
plt.plot(x_fit, y_fit, '--', color='gray', linewidth=3, alpha=0.6, label='Linear Fit')

plt.xlabel('Cyst Volume / Uterus Volume [%]', fontsize=14)
plt.ylabel('Normalized Luminescence Sum', fontsize=14)
plt.title('MRI vs. IVIS Luminescence', fontsize=16)
plt.colorbar(ScalarMappable(norm=Normalize(vmin=min(x), vmax=max(x)), cmap='viridis'),
             label='Cyst/Uterus Ratio [%]')
plt.grid(True)
plt.tight_layout()
plt.show()

TypeError: expected non-empty vector for x